In [240]:
import pandas as pd
import copy
import time

In [241]:
all_Gu = ['강남', '강동', '강북', '강서', '관악', '광진', '구로', '금천',
      '노원', '도봉', '동대문', '동작', '마포', '서대문', '서초', '성동', '성북', '송파',
      '양천', '영등포', '용산', '은평', '종로', '중', '중랑']

In [242]:
file_path1 = './인구+교통+POI+최단거리/cellid+인구+교통량+POI+최단거리_'  # 인구수, 교통량, 관심지점수, 가장 가까운 충전소까지의 거리
file_path2 = './cell중심간 최단거리/중심간 최단거리_'  # 각 cell의 중심간의 최단거리쌍
file_path3 = './결과/입지선정cell_'

exec_Gu = ['강남', '강동', '강북', '강서', '관악', '광진', '구로', '금천',
      '노원', '도봉', '동대문', '동작', '마포', '서대문', '서초', '성동', '성북', '송파',
      '양천', '영등포', '용산', '은평', '종로', '중', '중랑']  # 실행할 자치구의 집합

weights = {'인구':0.2, '교통량':0.4, 'POI':0.4}  # [총인구, 교통량, 관심지점수]를 입지매력점수에 반영할 가중치

num_new_stations = 4  # 추가할 충전소 의 개수

In [243]:
# 각 cell의 입지매력점수를 구해서 리턴하는 함수
def get_attr_score(df):
    df_temp = copy.deepcopy(df)
    # PERCENTILE RANK값에 가중치를 곱해서 입지매력점수 구하기
    df_temp['총인구_score'] = [r / len(df_temp) * weights['인구'] for r in df_temp['총인구'].rank(method='average')]
    df_temp['교통량_score'] = [r / len(df_temp) * weights['교통량'] for r in df_temp['교통량'].rank(method='average')]
    df_temp['관심지점수_score'] = [r / len(df_temp) * weights['POI'] for r in df_temp['관심지점수'].rank(method='min')]
    
    # 만점 10.0, 각 점수를 더해서 10점 만점으로 스케일한다
    #df['attr_score'] = [i+j for i in df_temp['총인구_score'] for j in df_temp['교통량_rank']]
    df['attr_score'] = df_temp[['총인구_score', '교통량_score', '관심지점수_score']].sum(axis=1) * 10
    
    return df
    

In [244]:
# cell별로 입지 잠재력 점수를 구하고 총합을 리턴한다.
# 잠재력 점수: 거리점수와 매력점수의 가중 평균
#               거리점수 * 가중치1 + 매력점수 * 가중치2
# 거리점수: 모든 cell의 중심에서 가장 가까운 충전소(가상충전소 포함)까지의 거리의 합을
#           원래상태의 거리 합에서 나눈 값
# 매력점수: 추가되는 충전소 cell들의 매력점수의 평균
def get_sum_pot_score(df, df_sd, cur_stations):
            
    # 거리점수 구하기
    # 1번 cell부터 돌면서
    # cell의 중심에서 가장 가까운 충전소까지의 거리를 새로 산정하고
    # 처음에 구해놓은 최단거리합(SUM_SD)에서 새로 계산된 거리합을 나눈다
    df_temp = copy.deepcopy(df)
    list_df = df['최단거리'].to_list()  # 시간단축을 위해 데이터프레임을 리스트로 변환한다
    list_df_sd = df_sd['total_cost'].to_list()  # 시간단축을 위해 데이터프레임을 리스트로 변환한다
    SUM_SD_cur = 0
    for i in range(len(df)):

        shortest_dist = list_df[i]
        for j in cur_stations:
            #dist = df_sd[(df_sd['origin_id']==i+1)&(df_sd['destination_id']== j+1)].iloc[0, 5]  # 'total_cost'를 가져온다
            #dist = df_sd.iloc[i * len(df) + j, 5]
            dist = list_df_sd[i * len(df) + j]
            if dist == 0:  # dist = 0일 경우는 자신과의 거리이므로 제외한다.
                continue
                
            #print('i: {}, j:{}   dist= :{}'.format(i, j, dist))
            if dist < shortest_dist:
                shortest_dist = dist
            
        #df_temp.loc[i, '최단거리'] = shortest_dist  # 갱신된 최단거리
        SUM_SD_cur += shortest_dist
 
    #SUM_SD_cur = df_temp['최단거리'].sum()  # 현재시점에서 각 cell의 중심에서 가장 가까운 충전소까지의 거리의 총합
    
    # 거리점수
    #dist_score = (SUM_SD / SUM_SD_cur - 1) * 100 # 최단거리합이 짧을 수록 점수가 높다
    dist_score = (1 - SUM_SD_cur / SUM_SD) * 40 # 최단거리합이 짧을 수록 점수가 높다
    
    # 매력점수: 후보cell들의 매력점수의 평균
    attr_score = df_temp.loc[cur_stations, ['attr_score']].mean()[0]
    
    total_score = dist_score + attr_score
    #print('거리점수: {:5.3f}   매력점수: {:5.3f}   잠재력점수: {:5.3f}'. format(dist_score, attr_score, total_score))
    
    return dist_score, attr_score, total_score

In [245]:
# 입지 잠재력점수가 최대가 되는 입지를 결정해서 리턴하는 함수
def get_optimum_stations(df, df_sd):
    # first_cells부터 시작해서 last_cells 에 도달할 때 까지 for 루프를 돈다
    # cell의 index는 0부터 시작한다
    first_cells = [i for i in range(num_new_stations)]  # 첫 n개의 cell
    last_cells = [len(df) -1 - i for i in reversed(range(num_new_stations))]   # 마지막 n개의 cell

    #print('first_cells = {}   last_cells = {}'.format(first_cells, last_cells))
    cur_stations = copy.deepcopy(first_cells)
    
    start = time.time()
    max_pot_score = 0
    max_pot_cells = [-1 for i in range(num_new_stations)]
    counter = 0
    while True:
        
        # get_sum_pot_score()를 불러 입지 잠재력점수의 합을 구한다
        d_score, a_score, cur_pot_score = get_sum_pot_score(df, df_sd, cur_stations)
        
        if cur_pot_score > max_pot_score:
            max_pot_score = cur_pot_score
            max_pot_cells = copy.deepcopy(cur_stations)
            print('counter: {:6d}  stations: {}  잠재력점수: {:5.3f}  거리점수: {:5.3f}  매력점수: {:5.3f}'.
                  format(counter, cur_stations, cur_pot_score, d_score, a_score))
    
        if counter % 1000 == 0:
            print('counter: {:10d}  {}'.format(counter, cur_stations))
            
        
        # last_cells에 도달하면 빠져 나간다
        if cur_stations == last_cells:
            break

        # n번째 충전소 위치를 다음 cell로 옮긴다.
        stay_flg = False
        for i in reversed(range(num_new_stations)):
            # 마지막 cell에 도착했으면 pass
            if cur_stations[i] == last_cells[i]:
                stay_flg = True
            else:
                cur_stations[i] = cur_stations[i] + 1
                
                # 마지막 충전소를 제외한 충전소의 위치를 옮겼을 때는
                # 그 이후의 충전소 cell의 위치를 초기화한다.
                if i < num_new_stations - 1:
                    for j in range(i + 1, num_new_stations):
                        cur_stations[j] = cur_stations[j-1] + 1
                    
                    stay_flg = False
        
            if not(stay_flg):
                break
            
        counter += 1
            
    print('counter = {}   last_cur_stations: {}   Elapsed time : {}분'.format(counter, cur_stations, round((time.time() - start)/60, 2)))
    print('최고 잠재력점수: {:5.3f}   선정된 cell: {}'.format(max_pot_score, max_pot_cells))
    
    return df

In [ ]:
for gu in exec_Gu:
    df_all = pd.read_csv(file_path1 + gu + '구.csv', engine='python', encoding='CP949')
    df_sd_cell = pd.read_csv(file_path2 + gu + '구.csv', engine='python', encoding='CP949')
    #df_all = df_all.fillna(0)
    
    print('*****'*10)
    print('*****', gu )
    print('*****'*10)
    SUM_SD = df_all['최단거리'].sum()  # 현재시점에서 각 cell의 중심에서 가장 가까운 충전소까지의 거리의 총합
    
    # 입지매력점수 구하기(10점 만점)
    df_all = get_attr_score(df_all)
    
    # 입지 잠재력점수가 최대가 되는 입지 선정(max_station의 수만큼 cell을 선택)
    df_all = get_optimum_stations(df_all, df_sd_cell)
    
    
    
    
    


**************************************************
***** 강남
**************************************************
counter:      0  stations: [0, 1, 2, 3]  잠재력점수: 5.625  거리점수: 2.443  매력점수: 3.182
counter:          0  [0, 1, 2, 3]
counter:      3  stations: [0, 1, 2, 6]  잠재력점수: 5.956  거리점수: 2.727  매력점수: 3.229
counter:     11  stations: [0, 1, 2, 14]  잠재력점수: 6.270  거리점수: 3.143  매력점수: 3.127
counter:     13  stations: [0, 1, 2, 16]  잠재력점수: 6.415  거리점수: 3.278  매력점수: 3.137
counter:     24  stations: [0, 1, 2, 27]  잠재력점수: 6.583  거리점수: 3.322  매력점수: 3.261
counter:     25  stations: [0, 1, 2, 28]  잠재력점수: 6.837  거리점수: 3.426  매력점수: 3.411
counter:     94  stations: [0, 1, 2, 97]  잠재력점수: 6.961  거리점수: 3.846  매력점수: 3.115
counter:    101  stations: [0, 1, 2, 104]  잠재력점수: 7.248  거리점수: 4.713  매력점수: 2.535
counter:    102  stations: [0, 1, 2, 105]  잠재력점수: 7.778  거리점수: 4.730  매력점수: 3.048
counter:    133  stations: [0, 1, 2, 136]  잠재력점수: 7.881  거리점수: 5.295  매력점수: 2.586
counter:    254  stations: [0, 1, 3, 104]  잠

In [195]:
df_all


,cellid,gid,총인구,도로수,교통량,관심지점수,최단거리,attr_score
0,1,다사57a46b,19.0,5.0,85705.0,0,1707.9,1.363057
1,2,다사57a47a,1188.0,7.0,316923.0,0,1334.9,2.942675
2,3,다사57b45a,159.0,8.0,295699.0,0,1111.8,2.636943
3,4,다사57b45b,1424.0,6.0,161896.0,2,1724.3,5.783439
4,5,다사57b46a,3262.0,3.0,110751.0,1,2136.7,5.261146
...,...,...,...,...,...,...,...,...
152,153,다사65b41b,5193.0,NaN,NaN,0,1025.7,1.464968
153,154,다사65b42a,902.0,2.0,37752.0,0,1275.2,0.815287
154,155,다사66a40b,7.0,1.0,34494.0,0,1052.3,0.471338
155,156,다사66a41a,0.0,2.0,21650.0,0,2740.2,0.261146


In [ ]:
df_all.loc[1, '최단거리']


In [385]:
aa = df_sd_cell[(df_sd_cell['origin_id']==1)&(df_sd_cell['destination_id']==3)]

In [397]:
aa.iloc[0, 1]

3

In [360]:
df_sd_cell.loc[0, 5]

0.0

In [283]:
df_all